# Шаг 2 - проведение расчётов и исследований

* Создать дополнительный категориальный столбец, который покажет количество рекламаций по каждому клиенту отдельно: мало (≤1), средне (=2), много (>2).
* Добавить категориальный столбец, разделив респондентов на три категории по возрасту: до 44 лет - молодой возраст, 45 - 59 лет - средний возраст, 60 лет и больше - пожилой возраст.

* Рассчитать средний возраст респондентов по полу и стране
* Рассчитать среднюю комфортную температуру в зависимости от возрастной категории
* Рассчитать медианное значение температуры и влажности для каждого типа охлаждения

* Добавить столбец с процентом удовлетворенных респондентов температурой воздуха в помещении относительно всех респондентов одной страны и одного пола

* Составить сводную таблицу, в которой данные будут сгруппированы по стране, полу, возрастной группе и подсчитаны средняя температура воздуха в помещении, на улице и средняя относительная влажность для каждой из этих групп.
* Самостоятельно найти информацию по стандартной комфортной относительной влажности в помещениях (в нашем случае - офисы). Создать новый категориальный столбец, в котором будет показано, удовлетворяет ли зафиксированная относительная влажность стандарту или нет. Категории назначить самостоятельно.
* Исследовать корреляцию между параметрами. Объяснить наиболее коррелирующие параметры, имеет ли это реальный смысл или просто особенность данных?


### Импорт датасета и либ

In [2]:
import seaborn as sns
import pandas as pd
import numpy as np


df = pd.read_csv('data_2.csv', delimiter=';')
df.head()

,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,пол,...,скорость_воздуха,рост,вес,занавески,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций
0,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,55.0,Женский,...,0.05,165.1,65.0,0,0,-1,-1,0,22.0,0
1,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,55.0,Женский,...,0.08,165.1,65.0,0,0,0,0,0,22.0,0
2,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,45.0,Женский,...,0.08,165.1,65.0,0,0,-1,-1,0,22.0,0
3,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,45.0,Женский,...,0.09,165.1,65.0,0,0,0,0,0,22.0,0
4,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,55.0,Женский,...,0.10,165.1,65.0,0,0,-1,-1,0,22.0,0


## Зачачи

### 1

### Добавить столбец с процентом удовлетворенных температурой (страна, пол)

In [15]:
# предпочтительное_изменение_температуры
df["температура_удовлетворительная_%"] = (df.groupby(["страна", "пол"], sort=False, dropna=False)["предпочтительное_изменение_температуры"].transform(
                                          lambda series: round(series.where(series == "Без изменений").count() / len(series) * 100, 1)))

for key, val in df.groupby(["страна", "пол"], sort=False, dropna=False)["предпочтительное_изменение_температуры"]:
    print(key, val.value_counts())

('Австралия', 'Женский') предпочтительное_изменение_температуры
Без изменений    50
Холоднее         29
Теплее           10
Name: count, dtype: int64
('Австралия', 'Мужской') предпочтительное_изменение_температуры
Без изменений    85
Холоднее         23
Теплее            1
Name: count, dtype: int64
('Индия', 'Женский') предпочтительное_изменение_температуры
Без изменений    34
Холоднее         21
Теплее           14
Name: count, dtype: int64
('Индия', 'Мужской') предпочтительное_изменение_температуры
Без изменений    83
Холоднее         40
Теплее           22
Name: count, dtype: int64
('США', nan) предпочтительное_изменение_температуры
Без изменений    36
Холоднее         23
Теплее           13
Name: count, dtype: int64


## Обзор полученного датасета (без графиков)

In [16]:
# Info
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484 entries, 0 to 483
Data columns (total 31 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   год                                          484 non-null    int64  
 1   время_года                                   484 non-null    object 
 2   климат                                       484 non-null    object 
 3   город                                        484 non-null    object 
 4   страна                                       484 non-null    object 
 5   способ_охлаждения                            484 non-null    object 
 6   режим_при_смешанном_типе_охлаждения          337 non-null    object 
 7   способ_обогрева                              222 non-null    object 
 8   возраст                                      484 non-null    float64
 9   пол                                          412 non-null    object 
 10  ощ

,год,возраст,ощущение_температуры,ощущение_температуры_(bool),ощущение_движения_воздуха_(bool),оценка_комфорта,утепление,температура_воздуха_в_помещении,температура_воздуха_на_улице,rh,...,рост,вес,занавески,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций,температура_удовлетворительная_%
count,484.000000,484.000000,484.000000,484.000000,484.000000,215.000000,484.000000,484.000000,484.000000,484.000000,...,484.000000,484.000000,484.000000,484.000000,484.000000,484.000000,484.000000,484.000000,484.000000,484.000000
mean,2011.033058,40.834711,0.203306,0.578512,0.206612,4.767442,0.634587,25.023140,24.100620,52.973760,...,165.556198,65.208678,0.004132,0.016529,-0.291322,-0.204545,0.092975,27.389669,0.105372,59.504050
std,0.922988,10.172098,1.091976,0.816518,0.605938,1.268549,0.253103,2.759505,0.231789,13.036204,...,5.242597,7.689621,0.386072,0.549549,0.701851,0.532096,0.393572,19.469562,0.440276,10.440719
min,2010.000000,22.000000,-3.000000,-1.000000,-1.000000,1.000000,0.360000,16.200000,22.800000,23.800000,...,134.600000,42.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,8.500000,0.000000,49.280000
25%,2010.000000,35.000000,0.000000,1.000000,0.000000,4.000000,0.460000,23.800000,24.100000,42.775000,...,165.100000,65.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,22.000000,0.000000,50.000000
50%,2011.000000,42.000000,0.000000,1.000000,0.000000,5.000000,0.590000,24.600000,24.100000,53.750000,...,165.100000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,0.000000,57.240000
75%,2012.000000,45.000000,1.000000,1.000000,1.000000,6.000000,0.700000,25.700000,24.100000,62.325000,...,165.100000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.800000,0.000000,57.240000
max,2012.000000,65.000000,3.000000,1.000000,1.000000,6.000000,2.080000,37.400000,25.900000,85.100000,...,190.500000,110.000000,1.000000,1.000000,1.000000,1.000000,1.000000,328.000000,3.000000,77.980000


In [17]:
# Value counts
for col in df.columns:
    print(col, df[col].value_counts(dropna=False))

год год
2012    214
2010    198
2011     72
Name: count, dtype: int64
время_года время_года
Лето     372
Зима      81
Весна     18
Осень     13
Name: count, dtype: int64
климат климат
Cубтропический океанический         270
Тропическая влажная саванна          77
Жаркий полузасушливый                70
Влажный субтропический муссонный     35
Субтропическое высокогорье           32
Name: count, dtype: int64
город город
Сидней        198
Техас          72
Ченнай         51
Ахмедабад      38
Дели           35
Хайдарабад     32
Шимла          32
Бангалор       26
Name: count, dtype: int64
страна страна
Индия        214
Австралия    198
США           72
Name: count, dtype: int64
способ_охлаждения способ_охлаждения
Смешанный            337
Кондиционирование    114
Вентиляция            33
Name: count, dtype: int64
режим_при_смешанном_типе_охлаждения режим_при_смешанном_типе_охлаждения
Кондиционирование    237
NaN                  147
Вентиляция           100
Name: count, dtype: int64
способ_

In [5]:
# Show sorted numeric
for col in df.select_dtypes(include='number').columns:
    vals = df[col]
    print(col, (vals.min(), vals.quantile(0.05), [vals.median(), vals.mean()],  vals.quantile(0.95), vals.max()),
          df[col].sort_values().tolist(), sep='\n', end='\n\n')

год
(2010, 2010.0, [2011.0, 2011.0330578512396], 2012.0, 2012)
[2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 

## Сохранение итогового датасета
Завершение 1 этапа и перехо ко 2 этапу работы

In [18]:
df.sort_values(["страна", "город", "год", "время_года", "способ_охлаждения", "режим_при_смешанном_типе_охлаждения", "пол", "оценка_комфорта"], inplace=True)
df.to_csv('data_3.csv', index=False, sep=';', encoding='utf-8')

## Заметки

1) 1
2) 2
3) 3
